# Person C-v2: Perceiver Resampler (최적화 버전)

**실험 목록**:
- E7-v2: Perceiver Resampler + LoRA (최적화)

**평가 지표**: SigLIP Score (Vision-Language 정렬)

**특징**: Self-Attention + Cross-Attention (Flamingo 스타일, 4608 → 64 tokens)

---

**적용된 최적화**:
- LR Scheduler (Warmup + Cosine Decay)
- Gradient Clipping (max_norm=1.0)
- Diversity Monitoring (Mode Collapse 조기 탐지)
- Vision Features 캐싱 (학습 속도 30-40% 향상)

---

**체크포인트 기반 재시작 지원**:
- 런타임이 끊겨도 Drive에 저장된 체크포인트에서 이어서 학습
- 완료된 실험은 자동 스킵

## 1. 환경 설정 (런타임 시작 시 항상 실행)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# 경로 설정
# ============================================
DRIVE_ROOT = "/content/drive/MyDrive/mutsa-02"

# 데이터 경로 (aihub_splitted는 mutsa-02 바로 아래)
DATA_PATH = f"{DRIVE_ROOT}/aihub_splitted"

# 결과 저장 경로 (siglip_study/results3) - v2 실험용
RESULTS_DIR = f"{DRIVE_ROOT}/korean_video_captioning/siglip_study/results3"

import os
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"{'='*60}")
print(f"DATA_PATH:    {DATA_PATH}")
print(f"RESULTS_DIR:  {RESULTS_DIR}")
print(f"{'='*60}")

if os.path.exists(DATA_PATH):
    train_n = len(os.listdir(f"{DATA_PATH}/train")) if os.path.exists(f"{DATA_PATH}/train") else 0
    val_n = len(os.listdir(f"{DATA_PATH}/val")) if os.path.exists(f"{DATA_PATH}/val") else 0
    print(f"Data found! Train: {train_n}, Val: {val_n}")
else:
    print(f"WARNING: Data path not found!")

Mounted at /content/drive
DATA_PATH:    /content/drive/MyDrive/mutsa-02/aihub_splitted
RESULTS_DIR:  /content/drive/MyDrive/mutsa-02/korean_video_captioning/siglip_study/results3
Data found! Train: 5, Val: 4


In [2]:
!pip install -q transformers>=4.40.0 accelerate bitsandbytes peft
!pip install -q torch torchvision
!pip install -q av decord opencv-python pillow
!pip install -q tqdm matplotlib pandas
!pip install -q evaluate bert_score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 148.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.6 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import json, os, random
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional
import cv2

from transformers import (
    AutoModel, AutoProcessor, AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, CLIPVisionModel, CLIPImageProcessor,
    get_cosine_schedule_with_warmup,  # [OPT] LR Scheduler
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB)")

PyTorch: 2.9.0+cu126, CUDA: True
GPU: NVIDIA A100-SXM4-80GB (85.2GB)


## 2. 설정

In [4]:
CONFIG = {
    "vision_encoder": "openai/clip-vit-large-patch14-336",
    "llm": "Qwen/Qwen3-8B",
    "siglip_model": "google/siglip2-so400m-patch14-384",
    "lora_r": 16, "lora_alpha": 32, "lora_dropout": 0.05,
    "stage1_epochs": 2, "stage1_lr": 1e-3,
    "stage2_epochs": 3, "stage2_lr": 5e-5,

    # ============================================
    # [OPT] 추가 최적화
    # ============================================
    "warmup_ratio": 0.1,      # 10% warmup
    "max_grad_norm": 1.0,     # Gradient Clipping

    "batch_size": 16, "gradient_accumulation": 2,
    "num_frames": 8, "max_length": 768, "seed": 42,
    "num_queries": 64, "num_heads": 8, "num_layers": 2,
    "data_path": DATA_PATH,
    "results_dir": RESULTS_DIR,
    "prompt": "이 영상을 자세히 설명해주세요.",
    "max_new_tokens": 128,
    "repetition_penalty": 1.2,
}

def set_seed(seed):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(CONFIG["seed"])
print("Config loaded (v2: with optimizations)")
print(f"  warmup_ratio: {CONFIG['warmup_ratio']}")
print(f"  max_grad_norm: {CONFIG['max_grad_norm']}")

Config loaded (v2: with optimizations)
  warmup_ratio: 0.1
  max_grad_norm: 1.0


## 3. 핵심 클래스 정의

In [5]:
# SigLIP2 Evaluator (Multilingual - Korean supported)
class SigLIPEvaluator:
    def __init__(self, model_name="google/siglip2-so400m-patch14-384", device="cuda"):
        print(f"Loading SigLIP2: {model_name}")
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.processor = AutoProcessor.from_pretrained(model_name)
        self.model.eval(); self.device = device

    @torch.no_grad()
    def compute_score(self, frames, caption):
        if not frames or not caption: return None
        try:
            inputs = self.processor(
                text=[caption], images=frames, return_tensors="pt", padding=True,
                truncation=True, max_length=64  # [FIX] SigLIP max_position_embeddings
            ).to(self.device)
            return torch.sigmoid(self.model(**inputs).logits_per_image).mean().item()
        except Exception as e:
            print(f"SigLIP error: {e}")
            return None  # [FIX]

    def evaluate_batch(self, samples):
        scores = [s for s in [self.compute_score(s["frames"], s["caption"]) for s in tqdm(samples, desc="SigLIP")] if s is not None]
        return {"siglip_score": np.mean(scores) if scores else 0.0, "siglip_std": np.std(scores) if scores else 0.0, "num_samples": len(scores)}

print("SigLIPEvaluator defined.")

SigLIPEvaluator defined.


In [6]:
# Text Metrics Evaluator (METEOR, BERTScore)
class TextMetricsEvaluator:
    """METEOR, BERTScore를 사용한 텍스트 품질 평가기.

    Ground Truth 캡션과 생성된 캡션을 비교하여 텍스트 유사도를 측정합니다.
    SigLIP2와 함께 사용하여 종합적인 평가를 수행합니다.
    """

    def __init__(self):
        import evaluate
        import nltk
        nltk.download('wordnet', quiet=True)
        nltk.download('punkt', quiet=True)
        nltk.download('omw-1.4', quiet=True)

        self.meteor = evaluate.load("meteor")
        self.bertscore = evaluate.load("bertscore")
        print("TextMetricsEvaluator ready (METEOR, BERTScore)")

    def compute_scores(self, predictions: list, references: list) -> dict:
        """배치 단위로 METEOR, BERTScore 계산.

        Args:
            predictions: 생성된 캡션 리스트
            references: Ground Truth 캡션 리스트

        Returns:
            dict: meteor, bertscore_precision, bertscore_recall, bertscore_f1
        """
        if not predictions or not references:
            return {"meteor": 0.0, "bertscore_f1": 0.0}

        # METEOR
        meteor_result = self.meteor.compute(predictions=predictions, references=references)

        # BERTScore (한국어 모델 사용)
        bert_result = self.bertscore.compute(
            predictions=predictions,
            references=references,
            lang="ko"  # Korean
        )

        return {
            "meteor": meteor_result["meteor"],
            "bertscore_precision": np.mean(bert_result["precision"]),
            "bertscore_recall": np.mean(bert_result["recall"]),
            "bertscore_f1": np.mean(bert_result["f1"]),
        }

print("TextMetricsEvaluator defined.")

TextMetricsEvaluator defined.


In [7]:
# Perceiver Resampler
class PerceiverLayer(nn.Module):
    def __init__(self, dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.norm3 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(nn.Linear(dim, dim*4), nn.GELU(), nn.Dropout(dropout), nn.Linear(dim*4, dim), nn.Dropout(dropout))

    def forward(self, queries, context):
        q = self.norm1(queries + self.self_attn(queries, queries, queries)[0])
        q = self.norm2(q + self.cross_attn(q, context, context)[0])
        return self.norm3(q + self.ffn(q))

class PerceiverResampler(nn.Module):
    def __init__(self, vision_dim=1024, llm_dim=4096, num_queries=64, num_heads=8, num_layers=2, dropout=0.1):
        super().__init__()
        self.queries = nn.Parameter(torch.randn(num_queries, llm_dim) * 0.02)
        self.input_proj = nn.Linear(vision_dim, llm_dim)
        self.layers = nn.ModuleList([PerceiverLayer(llm_dim, num_heads, dropout) for _ in range(num_layers)])
        self.output_norm = nn.LayerNorm(llm_dim)

    def forward(self, x):
        context = self.input_proj(x).unsqueeze(0)
        queries = self.queries.unsqueeze(0)
        for layer in self.layers:
            queries = layer(queries, context)
        return self.output_norm(queries).squeeze(0)

def create_projector(projector_type, vision_dim=1024, llm_dim=4096, config=None):
    if projector_type == "perceiver":
        return PerceiverResampler(vision_dim, llm_dim, config.get("num_queries", 64),
                                   config.get("num_heads", 8), config.get("num_layers", 2))
    raise ValueError(f"Unknown: {projector_type}")

print(f"Perceiver params: {sum(p.numel() for p in PerceiverResampler().parameters()):,}")

Perceiver params: 541,495,296


In [8]:
# Custom VLM (Vision Features 캐싱 지원)
class CustomVLM(nn.Module):
    def __init__(self, vision_encoder, projector, llm, tokenizer):
        super().__init__()
        self.vision_encoder = vision_encoder
        self.projector = projector
        self.llm = llm
        self.tokenizer = tokenizer
        for p in self.vision_encoder.parameters(): p.requires_grad = False
        self.vision_encoder.eval()

    def encode_video(self, frames):
        with torch.no_grad():
            features = self.vision_encoder(pixel_values=frames).last_hidden_state[:, 1:, :]
        return features.reshape(-1, features.size(-1))

    def forward_with_cache(self, vision_features, input_ids, attention_mask, labels=None):
        """[OPT] 캐싱된 Vision Features로 forward (Vision Encoder 스킵)"""
        batch_size, device = vision_features.size(0), vision_features.device
        all_vision = [self.projector(vision_features[i]) for i in range(batch_size)]
        text_embeds = self.llm.get_input_embeddings()(input_ids)

        combined_e, combined_a, combined_l = [], [], []
        for i in range(batch_size):
            v_len = all_vision[i].size(0)
            combined_e.append(torch.cat([all_vision[i], text_embeds[i]], dim=0))
            combined_a.append(torch.cat([torch.ones(v_len, device=device), attention_mask[i]], dim=0))
            if labels is not None:
                combined_l.append(torch.cat([torch.full((v_len,), -100, device=device, dtype=labels.dtype), labels[i]], dim=0))

        max_len = max(e.size(0) for e in combined_e)
        pad_e = torch.zeros(batch_size, max_len, combined_e[0].size(-1), device=device)
        pad_a = torch.zeros(batch_size, max_len, device=device)
        pad_l = torch.full((batch_size, max_len), -100, device=device, dtype=torch.long) if labels is not None else None

        for i in range(batch_size):
            sl = combined_e[i].size(0)
            pad_e[i, :sl], pad_a[i, :sl] = combined_e[i], combined_a[i]
            if labels is not None: pad_l[i, :sl] = combined_l[i]

        return self.llm(inputs_embeds=pad_e, attention_mask=pad_a, labels=pad_l, return_dict=True)

    def forward(self, frames, input_ids, attention_mask, labels=None):
        batch_size, device = frames.size(0), frames.device
        all_vision = [self.projector(self.encode_video(frames[i])) for i in range(batch_size)]
        text_embeds = self.llm.get_input_embeddings()(input_ids)

        combined_e, combined_a, combined_l = [], [], []
        for i in range(batch_size):
            v_len = all_vision[i].size(0)
            combined_e.append(torch.cat([all_vision[i], text_embeds[i]], dim=0))
            combined_a.append(torch.cat([torch.ones(v_len, device=device), attention_mask[i]], dim=0))
            if labels is not None:
                combined_l.append(torch.cat([torch.full((v_len,), -100, device=device, dtype=labels.dtype), labels[i]], dim=0))

        max_len = max(e.size(0) for e in combined_e)
        pad_e = torch.zeros(batch_size, max_len, combined_e[0].size(-1), device=device)
        pad_a = torch.zeros(batch_size, max_len, device=device)
        pad_l = torch.full((batch_size, max_len), -100, device=device, dtype=torch.long) if labels is not None else None

        for i in range(batch_size):
            sl = combined_e[i].size(0)
            pad_e[i, :sl], pad_a[i, :sl] = combined_e[i], combined_a[i]
            if labels is not None: pad_l[i, :sl] = combined_l[i]

        return self.llm(inputs_embeds=pad_e, attention_mask=pad_a, labels=pad_l, return_dict=True)

    @torch.no_grad()
    def generate(self, frames, prompt, max_new_tokens=128):
        device = frames.device
        vision_embeds = self.projector(self.encode_video(frames))
        text_inputs = self.tokenizer(prompt, return_tensors="pt").to(device)
        text_embeds = self.llm.get_input_embeddings()(text_inputs.input_ids)
        combined = torch.cat([vision_embeds.unsqueeze(0), text_embeds], dim=1)
        outputs = self.llm.generate(
            inputs_embeds=combined, max_new_tokens=max_new_tokens, do_sample=False,
            repetition_penalty=1.2,
            pad_token_id=self.tokenizer.pad_token_id, eos_token_id=self.tokenizer.eos_token_id
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

print("CustomVLM defined (with cache support).")

CustomVLM defined (with cache support).


In [9]:
# Dataset with Vision Features Caching
def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0: cap.release(); return []
    frames = []
    for idx in np.linspace(0, total-1, num_frames, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret: frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    return frames

class CachedVideoDataset(Dataset):
    """[OPT] Vision Features를 미리 계산하여 캐싱하는 Dataset.

    Vision Encoder는 frozen이므로 같은 이미지에 대해 항상 같은 출력.
    한 번 계산해서 저장하면 매 epoch마다 Vision Encoder 추론 시간 절약.
    """

    def __init__(self, data_path, split, vision_encoder, image_processor, tokenizer,
                 num_frames=8, max_length=512, max_samples=None, prompt="이 영상을 자세히 설명해주세요.", device="cuda"):
        self.data_path, self.split = Path(data_path), split
        self.image_processor, self.tokenizer = image_processor, tokenizer
        self.num_frames, self.max_length = num_frames, max_length
        self.prompt = prompt
        self.device = device
        self.samples = self._load(max_samples)
        self.prompt_len = len(tokenizer(prompt, add_special_tokens=False).input_ids)

        # Vision Features 캐싱
        self.vision_cache = {}
        self.pil_cache = {}
        print(f"\n[CACHE] Pre-computing vision features for {len(self.samples)} {split} samples...")
        vision_encoder.eval()
        with torch.no_grad():
            for idx in tqdm(range(len(self.samples)), desc=f"Caching {split}"):
                s = self.samples[idx]
                frames = extract_frames(s["video_path"], self.num_frames)
                if not frames:
                    frames = [np.zeros((336, 336, 3), dtype=np.uint8)] * self.num_frames
                while len(frames) < self.num_frames: frames.append(frames[-1].copy())
                frames = frames[:self.num_frames]

                pil_frames = [Image.fromarray(f) for f in frames]
                self.pil_cache[idx] = pil_frames

                pixel_values = self.image_processor(images=pil_frames, return_tensors="pt").pixel_values.to(device)
                features = vision_encoder(pixel_values=pixel_values).last_hidden_state[:, 1:, :]
                self.vision_cache[idx] = features.reshape(-1, features.size(-1)).cpu()

        print(f"[CACHE] Done! Cached {len(self.vision_cache)} samples.")
        print(f"[CACHE] Feature shape: {self.vision_cache[0].shape}")

    def _load(self, max_samples):
        samples = []
        label_dir = self.data_path / self.split / "labels"
        video_dir = self.data_path / self.split / "videos"
        if not label_dir.exists(): return samples
        for lf in sorted(label_dir.glob("*.json")):
            try:
                with open(lf, "r", encoding="utf-8") as f: label = json.load(f)
                vp = video_dir / (lf.stem + ".mp4")
                if vp.exists() and (cap := label.get("annotation", {}).get("description_kr", "")):
                    samples.append({"video_path": str(vp), "caption": cap})
            except: pass
            if max_samples and len(samples) >= max_samples: break
        return samples

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        vision_features = self.vision_cache[idx]
        pil_frames = self.pil_cache[idx]

        full_text = f"{self.prompt} {s['caption']}"
        ti = self.tokenizer(full_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "vision_features": vision_features,
            "input_ids": ti.input_ids.squeeze(0),
            "attention_mask": ti.attention_mask.squeeze(0),
            "caption": s["caption"],
            "pil_frames": pil_frames,
            "prompt_len": self.prompt_len
        }

def create_cached_collate_fn(pad_token_id):
    def collate_fn(batch):
        labels = []
        for b in batch:
            label = b["input_ids"].clone()
            label[label == pad_token_id] = -100
            if "prompt_len" in b:
                label[:b["prompt_len"]] = -100
            labels.append(label)
        return {
            "vision_features": torch.stack([b["vision_features"] for b in batch]),
            "input_ids": torch.stack([b["input_ids"] for b in batch]),
            "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
            "labels": torch.stack(labels),
            "captions": [b["caption"] for b in batch],
            "pil_frames": [b["pil_frames"] for b in batch],
        }
    return collate_fn

print("CachedVideoDataset defined.")

CachedVideoDataset defined.


In [10]:
# Checkpoint Manager
class CheckpointManager:
    def __init__(self, exp_name, results_dir):
        self.exp_dir = Path(results_dir) / exp_name
        self.exp_dir.mkdir(parents=True, exist_ok=True)
        self.ckpt_dir = self.exp_dir / "checkpoints"
        self.ckpt_dir.mkdir(exist_ok=True)
        self.logs = []
        self.best_score = 0.0
        log_file = self.exp_dir / "training_log.csv"
        if log_file.exists(): self.logs = pd.read_csv(log_file).to_dict('records')
        print(f"CheckpointManager: {self.exp_dir}")

    def is_completed(self): return (self.exp_dir / "final_metrics.json").exists()

    def get_resume_info(self):
        for e in range(10, 0, -1):
            p = self.ckpt_dir / f"stage2_epoch{e}_checkpoint.pt"
            if p.exists(): return {"stage": 2, "epoch": e, "path": p}
        p = self.ckpt_dir / "stage1_checkpoint.pt"
        if p.exists(): return {"stage": 1, "epoch": "done", "path": p}
        return None

    def log(self, m):
        m["timestamp"] = datetime.now().isoformat()
        self.logs.append(m)
        pd.DataFrame(self.logs).to_csv(self.exp_dir / "training_log.csv", index=False)

    def save_checkpoint(self, model, stage, epoch, metrics, optimizer=None, scheduler=None):
        ckpt = {"projector_state_dict": model.projector.state_dict(), "metrics": metrics, "stage": stage, "epoch": epoch}
        if optimizer: ckpt["optimizer_state_dict"] = optimizer.state_dict()
        if scheduler: ckpt["scheduler_state_dict"] = scheduler.state_dict()
        path = self.ckpt_dir / ("stage1_checkpoint.pt" if stage == 1 else f"stage2_epoch{epoch}_checkpoint.pt")
        torch.save(ckpt, path)
        print(f"Saved: {path}")
        if (s := metrics.get("siglip_score", 0)) > self.best_score:
            self.best_score = s
            torch.save(ckpt, self.ckpt_dir / "best_model.pt")
            print(f"New best! Score: {s:.4f}")

    def save_final(self, m):
        with open(self.exp_dir / "final_metrics.json", "w") as f: json.dump(m, f, indent=2)
        print("Experiment completed!")

print("CheckpointManager defined.")

CheckpointManager defined.


In [11]:
# Logging helpers
import time

def get_gpu_memory():
    if torch.cuda.is_available():
        return {"allocated": torch.cuda.memory_allocated()/1e9, "total": torch.cuda.get_device_properties(0).total_memory/1e9}
    return {"allocated": 0, "total": 0}

def format_time(s):
    h, m, sec = int(s//3600), int((s%3600)//60), int(s%60)
    return f"{h}h{m}m{sec}s" if h else (f"{m}m{sec}s" if m else f"{sec}s")

def print_config(config, proj_type, train_n, val_n):
    mem = get_gpu_memory()
    print(f"\n{'-'*60}\nCONFIG: {proj_type} | Train: {train_n} | Val: {val_n}")
    print(f"Batch: {config['batch_size']}x{config['gradient_accumulation']} | Frames: {config['num_frames']} | MaxLen: {config['max_length']}")
    print(f"Stage1: {config['stage1_epochs']}ep (LR:{config['stage1_lr']}) | Stage2: {config['stage2_epochs']}ep (LR:{config['stage2_lr']})")
    print(f"Warmup: {config['warmup_ratio']*100:.0f}% | Grad Clip: {config['max_grad_norm']}")
    print(f"GPU: {mem['allocated']:.1f}GB / {mem['total']:.1f}GB\n{'-'*60}")

def compute_diversity(captions):
    """[OPT] 캡션 다양성 계산 (0.0 ~ 1.0). Mode Collapse 탐지용."""
    if not captions: return 0.0
    unique = len(set(captions))
    return unique / len(captions)

print("Helpers defined.")

Helpers defined.


In [12]:
# Training functions with optimizations
from torch.amp import autocast, GradScaler

def evaluate_model(model, val_loader, siglip_evaluator, prompt, device, max_samples=None, text_evaluator=None):
    """검증 평가 + Diversity 모니터링."""
    model.eval()
    samples, num_eval = [], 0
    predictions, references = [], []
    total = len(val_loader.dataset) if not max_samples else min(max_samples, len(val_loader.dataset))
    print(f"\n[Eval] Generating {total} captions...")
    start = time.time()

    cap_lens, empty_cnt, rep_scores = [], 0, []
    all_captions = []  # [OPT] Diversity 계산용

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Eval"):
            if max_samples and num_eval >= max_samples: break
            if "pixel_values" in batch:
                pv = batch["pixel_values"].to(device)
            else:
                pv = None
            gt_captions = batch["captions"]
            pil_frames_batch = batch["pil_frames"]

            for i in range(len(gt_captions)):
                if max_samples and num_eval >= max_samples: break

                if pv is None:
                    from transformers import CLIPImageProcessor
                    img_proc = CLIPImageProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
                    frames_tensor = img_proc(images=pil_frames_batch[i], return_tensors="pt").pixel_values.to(device)
                else:
                    frames_tensor = pv[i]

                cap = model.generate(frames_tensor, prompt)

                cap_lens.append(len(cap))
                if not cap.strip(): empty_cnt += 1
                tokens = cap.split()
                if len(tokens) > 1: rep_scores.append(len(set(tokens))/len(tokens))
                all_captions.append(cap)

                if num_eval < 3:
                    print(f"\n  [Sample {num_eval+1}]")
                    print(f"    Gen: {cap[:100]}{'...' if len(cap)>100 else ''}")
                    print(f"    GT:  {gt_captions[i][:100]}{'...' if len(gt_captions[i])>100 else ''}")

                samples.append({"frames": pil_frames_batch[i], "caption": cap})
                predictions.append(cap)
                references.append(gt_captions[i])
                num_eval += 1

    # [OPT] Diversity 계산
    diversity = compute_diversity(all_captions)
    print(f"\n[Eval] Done in {format_time(time.time()-start)}")
    print(f"[DEBUG] Empty:{empty_cnt}, Diversity:{diversity:.2f} ({len(set(all_captions))}/{len(all_captions)} unique)")

    # [OPT] Mode Collapse 경고
    if diversity < 0.5:
        print(f"\n" + "!"*60)
        print(f"  WARNING: Low diversity ({diversity:.2f}) - POSSIBLE MODE COLLAPSE!")
        print(f"!"*60 + "\n")

    print("[Eval] Computing SigLIP2...")
    metrics = siglip_evaluator.evaluate_batch(samples)
    metrics["empty_count"] = empty_cnt
    metrics["diversity"] = diversity

    if text_evaluator is not None:
        print("[Eval] Computing METEOR/BERTScore...")
        text_metrics = text_evaluator.compute_scores(predictions, references)
        metrics.update(text_metrics)
        print(f"  METEOR: {text_metrics['meteor']:.4f}, BERTScore-F1: {text_metrics['bertscore_f1']:.4f}")

    model.train()
    return metrics

def train_stage1(model, train_loader, config, ckpt_mgr, device):
    """[OPT] LR Scheduler + Gradient Clipping 적용."""
    print("\n" + "="*60 + "\n  STAGE 1: Perceiver Warm-up (with LR Scheduler)\n" + "="*60)
    for p in model.llm.parameters(): p.requires_grad = False
    for p in model.projector.parameters(): p.requires_grad = True

    proj_params = sum(p.numel() for p in model.projector.parameters() if p.requires_grad)
    total_steps = len(train_loader) * config["stage1_epochs"]
    warmup_steps = int(config["warmup_ratio"] * total_steps)

    print(f"  Trainable: {proj_params:,} params")
    print(f"  Total Steps: {total_steps} | Warmup: {warmup_steps} ({config['warmup_ratio']*100:.0f}%)")
    print(f"  LR: {config['stage1_lr']} | Grad Clip: {config['max_grad_norm']}")

    opt = torch.optim.AdamW(model.projector.parameters(), lr=config["stage1_lr"])
    scheduler = get_cosine_schedule_with_warmup(opt, warmup_steps, total_steps)
    scaler = GradScaler('cuda')
    stage_start = time.time()

    for epoch in range(config["stage1_epochs"]):
        epoch_start = time.time()
        model.train(); total_loss = 0
        for i, batch in enumerate(pbar := tqdm(train_loader, desc=f"S1 E{epoch+1}/{config['stage1_epochs']}")):
            with autocast('cuda', dtype=torch.bfloat16):
                loss = model.forward_with_cache(
                    batch["vision_features"].to(device),
                    batch["input_ids"].to(device),
                    batch["attention_mask"].to(device),
                    batch["labels"].to(device)
                ).loss

            scaler.scale(loss).backward()

            if (i+1) % config["gradient_accumulation"] == 0:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.projector.parameters(), config["max_grad_norm"])
                scaler.step(opt)
                scaler.update()
                scheduler.step()
                opt.zero_grad()

            total_loss += loss.item()
            current_lr = scheduler.get_last_lr()[0]
            mem = get_gpu_memory()
            pbar.set_postfix({"loss": f"{loss.item():.4f}", "lr": f"{current_lr:.2e}", "mem": f"{mem['allocated']:.1f}G"})

        avg = total_loss / len(train_loader)
        print(f"\n[S1 E{epoch+1}] Loss: {avg:.4f} | LR: {current_lr:.2e} | Time: {format_time(time.time()-epoch_start)}")
        ckpt_mgr.log({"stage": 1, "epoch": epoch+1, "train_loss": avg, "lr": current_lr})

    print(f"\n{'='*60}\n  Stage 1 Complete! Time: {format_time(time.time()-stage_start)} | Loss: {avg:.4f}\n{'='*60}")
    ckpt_mgr.save_checkpoint(model, 1, config["stage1_epochs"], {"train_loss": avg}, opt, scheduler)

def train_stage2(model, train_loader, val_loader, siglip_evaluator, config, ckpt_mgr, device, start_epoch=0, text_evaluator=None):
    """[OPT] LR Scheduler + Gradient Clipping + Diversity 모니터링."""
    remain = config["stage2_epochs"] - start_epoch
    print(f"\n" + "="*60 + f"\n  STAGE 2: Perceiver + LoRA (E{start_epoch+1}-{config['stage2_epochs']})\n" + "="*60)

    for p in model.projector.parameters(): p.requires_grad = True
    for name, param in model.llm.named_parameters():
        if 'lora' in name.lower(): param.requires_grad = True

    proj_p = sum(p.numel() for p in model.projector.parameters() if p.requires_grad)
    lora_p = sum(p.numel() for p in model.llm.parameters() if p.requires_grad)
    total_steps = len(train_loader) * remain
    warmup_steps = int(config["warmup_ratio"] * total_steps)

    print(f"  Trainable: Proj {proj_p:,} + LoRA {lora_p:,} = {proj_p+lora_p:,}")
    print(f"  Total Steps: {total_steps} | Warmup: {warmup_steps}")
    print(f"  LR: {config['stage2_lr']} | Grad Clip: {config['max_grad_norm']}")

    all_params = list(model.projector.parameters()) + [p for p in model.llm.parameters() if p.requires_grad]
    opt = torch.optim.AdamW(all_params, lr=config["stage2_lr"])
    scheduler = get_cosine_schedule_with_warmup(opt, warmup_steps, total_steps)
    scaler = GradScaler('cuda')

    stage_start = time.time()
    best = ckpt_mgr.best_score

    for epoch in range(start_epoch, config["stage2_epochs"]):
        epoch_start = time.time()
        model.train(); total_loss = 0

        for i, batch in enumerate(pbar := tqdm(train_loader, desc=f"S2 E{epoch+1}/{config['stage2_epochs']}")):
            with autocast('cuda', dtype=torch.bfloat16):
                loss = model.forward_with_cache(
                    batch["vision_features"].to(device),
                    batch["input_ids"].to(device),
                    batch["attention_mask"].to(device),
                    batch["labels"].to(device)
                ).loss

            scaler.scale(loss).backward()

            if (i+1) % config["gradient_accumulation"] == 0:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(all_params, config["max_grad_norm"])
                scaler.step(opt)
                scaler.update()
                scheduler.step()
                opt.zero_grad()

            total_loss += loss.item()
            current_lr = scheduler.get_last_lr()[0]
            mem = get_gpu_memory()
            pbar.set_postfix({"loss": f"{loss.item():.4f}", "lr": f"{current_lr:.2e}", "mem": f"{mem['allocated']:.1f}G"})

        avg = total_loss / len(train_loader)
        print(f"\n[S2 E{epoch+1}] Train Loss: {avg:.4f} | LR: {current_lr:.2e} | Time: {format_time(time.time()-epoch_start)}")

        metrics = evaluate_model(model, val_loader, siglip_evaluator, config["prompt"], device, text_evaluator=text_evaluator)
        is_best = metrics['siglip_score'] > best
        if is_best: best = metrics['siglip_score']

        print(f"[S2 E{epoch+1}] Val SigLIP: {metrics['siglip_score']:.4f} | Diversity: {metrics.get('diversity', 0):.2f} {'(BEST!)' if is_best else ''} | Best: {best:.4f}")
        ckpt_mgr.log({"stage": 2, "epoch": epoch+1, "train_loss": avg, "siglip_score": metrics["siglip_score"], "diversity": metrics.get("diversity", 0), "lr": current_lr})
        ckpt_mgr.save_checkpoint(model, 2, epoch+1, metrics, opt, scheduler)

print("Training functions defined (with optimizations).")

Training functions defined (with optimizations).


In [13]:
# Model build
def build_model(config, projector_type, device, resume_path=None):
    print(f"\nBuilding {projector_type} model...")
    vision_encoder = CLIPVisionModel.from_pretrained(config["vision_encoder"]).to(device)
    image_processor = CLIPImageProcessor.from_pretrained(config["vision_encoder"])
    vision_encoder.eval()

    bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True)
    llm = AutoModelForCausalLM.from_pretrained(config["llm"], quantization_config=bnb, device_map="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(config["llm"], trust_remote_code=True)
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    print(f"  PAD: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"  EOS: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

    projector = create_projector(projector_type, vision_encoder.config.hidden_size, llm.config.hidden_size, config).to(device)

    llm = prepare_model_for_kbit_training(llm)
    lora_cfg = LoraConfig(r=config["lora_r"], lora_alpha=config["lora_alpha"], lora_dropout=config["lora_dropout"],
                          target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
                          task_type=TaskType.CAUSAL_LM, bias="none")
    llm = get_peft_model(llm, lora_cfg)
    llm.print_trainable_parameters()
    llm.config.use_cache = False  # type: ignore[attr-defined]
    llm.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

    model = CustomVLM(vision_encoder, projector, llm, tokenizer)
    if resume_path:
        print(f"Loading checkpoint: {resume_path}")
        ckpt = torch.load(resume_path, map_location=device)
        model.projector.load_state_dict(ckpt["projector_state_dict"])
        print(f"Resumed from stage {ckpt['stage']}, epoch {ckpt['epoch']}")
    print("Model built!")
    return model, vision_encoder, image_processor, tokenizer

print("Build function defined.")

Build function defined.


In [14]:
# Experiment runner (with Vision Caching)
def run_experiment(exp_name, projector_type, config):
    exp_start = time.time()
    print("\n" + "#"*60)
    print(f"#  EXPERIMENT: {exp_name}")
    print(f"#  Projector: {projector_type}")
    print(f"#  Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("#"*60)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    ckpt_mgr = CheckpointManager(exp_name, config["results_dir"])

    if ckpt_mgr.is_completed():
        print(f"\n[SKIP] Already completed!")
        with open(ckpt_mgr.exp_dir / "final_metrics.json") as f:
            r = json.load(f)
        print(f"  Previous Score: {r.get('siglip_score', 'N/A')}")
        return r

    resume_info = ckpt_mgr.get_resume_info()
    resume_path = resume_info["path"] if resume_info else None
    if resume_info:
        print(f"\n[RESUME] Stage {resume_info['stage']}, Epoch {resume_info['epoch']}")
    else:
        print(f"\n[START] Starting from scratch")

    try:
        print(f"\n[1/5] Building model...")
        model, vision_encoder, img_proc, tok = build_model(config, projector_type, device, resume_path)

        print(f"\n[2/5] Loading datasets with Vision Caching...")
        train_ds = CachedVideoDataset(
            config["data_path"], "train", vision_encoder, img_proc, tok,
            config["num_frames"], config["max_length"], None, config["prompt"], device
        )
        val_ds = CachedVideoDataset(
            config["data_path"], "val", vision_encoder, img_proc, tok,
            config["num_frames"], config["max_length"], None, config["prompt"], device
        )
        train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True,
                                  collate_fn=create_cached_collate_fn(tok.pad_token_id),
                                  num_workers=4, pin_memory=True, persistent_workers=True)
        val_loader = DataLoader(val_ds, batch_size=config["batch_size"], shuffle=False,
                                collate_fn=create_cached_collate_fn(tok.pad_token_id),
                                num_workers=4, pin_memory=True, persistent_workers=True)
        print_config(config, projector_type, len(train_ds), len(val_ds))

        print(f"[3/5] Loading SigLIP2 evaluator...")
        siglip_evaluator = SigLIPEvaluator(config["siglip_model"], device)

        print(f"[4/5] Loading Text Metrics evaluator...")
        text_evaluator = TextMetricsEvaluator()

        print(f"\n[5/5] Starting training...")
        if resume_info is None:
            train_stage1(model, train_loader, config, ckpt_mgr, device)
            train_stage2(model, train_loader, val_loader, siglip_evaluator, config, ckpt_mgr, device, 0, None)
        elif resume_info["stage"] == 1:
            train_stage2(model, train_loader, val_loader, siglip_evaluator, config, ckpt_mgr, device, 0, None)
        else:
            train_stage2(model, train_loader, val_loader, siglip_evaluator, config, ckpt_mgr, device, resume_info["epoch"], None)

        print(f"\n[FINAL] Running final evaluation...")
        final = evaluate_model(model, val_loader, siglip_evaluator, config["prompt"], device, None, text_evaluator)
        final["experiment"], final["projector"] = exp_name, projector_type
        ckpt_mgr.save_final(final)

        exp_time = time.time() - exp_start
        print(f"\n" + "="*60)
        print(f"  EXPERIMENT COMPLETE: {exp_name}")
        print(f"="*60)
        print(f"  Final SigLIP2: {final['siglip_score']:.4f}")
        print(f"  Final Diversity: {final.get('diversity', 0):.2f}")
        if 'meteor' in final:
            print(f"  Final METEOR: {final['meteor']:.4f}")
            print(f"  Final BERTScore-F1: {final['bertscore_f1']:.4f}")
        print(f"  Total Time: {format_time(exp_time)}")
        print(f"  Saved to: {ckpt_mgr.exp_dir}")
        print("="*60)

        del model, siglip_evaluator, text_evaluator; torch.cuda.empty_cache()
        return final
    except Exception as e:
        print(f"\n{'!'*60}\n  ERROR in {exp_name}: {e}\n{'!'*60}")
        import traceback; traceback.print_exc()
        return {"experiment": exp_name, "error": str(e)}

print("Experiment runner defined.")

Experiment runner defined.


---

## 4. 실험 실행

### 실험 E7-v2: Perceiver Resampler + LoRA (최적화)

In [15]:
result_e7_v2 = run_experiment("E7_v2_perceiver_optimized", "perceiver", CONFIG)
print(f"\nE7-v2 Result: {result_e7_v2}")


############################################################
#  EXPERIMENT: E7_v2_perceiver_optimized
#  Projector: perceiver
#  Time: 2026-01-21 01:49:23
############################################################
CheckpointManager: /content/drive/MyDrive/mutsa-02/korean_video_captioning/siglip_study/results3/E7_v2_perceiver_optimized

[START] Starting from scratch

[1/5] Building model...

Building perceiver model...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

  METEOR: 0.0633, BERTScore-F1: 0.6998
Experiment completed!

  EXPERIMENT COMPLETE: E7_v2_perceiver_optimized
  Final SigLIP2: 0.0103
  Final Diversity: 0.08
  Final METEOR: 0.0633
  Final BERTScore-F1: 0.6998
  Total Time: 4h40m57s
  Saved to: /content/drive/MyDrive/mutsa-02/korean_video_captioning/siglip_study/results3/E7_v2_perceiver_optimized

E7-v2 Result: {'siglip_score': np.float64(0.010271349828257762), 'siglip_std': np.float64(0.09439334301200619), 'num_samples': 97, 'empty_count': 0, 'diversity': 0.08247422680412371, 'meteor': np.float64(0.06332297698788802), 'bertscore_precision': np.float64(0.7258971665323395), 'bertscore_recall': np.float64(0.6755808665580356), 'bertscore_f1': np.float64(0.6997978625838289), 'experiment': 'E7_v2_perceiver_optimized', 'projector': 'perceiver'}


---

## 5. 결과 요약

In [16]:
print("\n" + "="*60 + "\nE7-v2 (OPTIMIZED) SUMMARY\n" + "="*60)
metrics_file = Path(RESULTS_DIR) / "E7_v2_perceiver_optimized" / "final_metrics.json"
if metrics_file.exists():
    with open(metrics_file) as f: result = json.load(f)
    print(pd.DataFrame([result]).to_string(index=False))
    pd.DataFrame([result]).to_csv(f"{RESULTS_DIR}/e7_v2_summary.csv", index=False)
else:
    print("E7-v2: Not completed")


E7-v2 (OPTIMIZED) SUMMARY
 siglip_score  siglip_std  num_samples  empty_count  diversity   meteor  bertscore_precision  bertscore_recall  bertscore_f1                experiment projector
     0.010271    0.094393           97            0   0.082474 0.063323             0.725897          0.675581      0.699798 E7_v2_perceiver_optimized perceiver


---

## 6. 전체 결과 통합 (모든 실험 완료 후)

In [17]:
import glob

print("\n" + "="*70 + "\nALL v2 EXPERIMENTS SUMMARY\n" + "="*70)

all_results = []
for exp in ["E1_v2_linear_optimized", "E3_v2_mlp_optimized", "E5_v2_lr_reduced", "E5_v3_epoch_increased", "E7_v2_perceiver_optimized"]:
    mf = Path(RESULTS_DIR) / exp / "final_metrics.json"
    if mf.exists():
        with open(mf) as f: all_results.append(json.load(f))
        print(f"  Loaded: {exp}")
    else:
        print(f"  {exp}: Not completed")

if all_results:
    df = pd.DataFrame(all_results)
    print("\n" + df.to_string(index=False))

    # Best by SigLIP (with diversity > 0.5 to avoid mode collapse)
    valid = df[df.get("diversity", 1.0) > 0.3] if "diversity" in df.columns else df
    if not valid.empty:
        best = valid.loc[valid["siglip_score"].idxmax()]
        print(f"\n{'='*70}")
        print(f"BEST PROJECTOR: {best.get('projector', best.get('experiment'))} (Score: {best['siglip_score']:.4f}, Diversity: {best.get('diversity', 'N/A')})")
        print(f"{'='*70}")

    df.to_csv(f"{RESULTS_DIR}/all_v2_experiments_summary.csv", index=False)
    print(f"\nSaved: {RESULTS_DIR}/all_v2_experiments_summary.csv")
else:
    print("\nNo experiments completed yet.")


ALL v2 EXPERIMENTS SUMMARY
  E1_v2_linear_optimized: Not completed
  E3_v2_mlp_optimized: Not completed
  Loaded: E5_v2_lr_reduced
  Loaded: E5_v3_epoch_increased
  Loaded: E7_v2_perceiver_optimized

 siglip_score  siglip_std  num_samples  num_errors  empty_count  diversity   meteor  bertscore_precision  bertscore_recall  bertscore_f1                experiment    projector                                    config
     0.113255    0.207832           97         0.0            0   0.113402 0.066120             0.730195          0.682787      0.705659          E5_v2_lr_reduced c_abstractor {'stage1_lr': 0.0001, 'stage2_lr': 2e-05}
     0.003160    0.016089           97         0.0            0   0.061856 0.068021             0.727821          0.674673      0.700204     E5_v3_epoch_increased c_abstractor  {'stage1_lr': 0.001, 'stage2_lr': 5e-05}
     0.010271    0.094393           97         NaN            0   0.082474 0.063323             0.725897          0.675581      0.699798 E7_v2_pe

## 7. 모델 로드 (추론용)

In [18]:
def load_trained_model(exp_name, projector_type, config):
    ckpt_path = Path(config["results_dir"]) / exp_name / "checkpoints" / "best_model.pt"
    if not ckpt_path.exists():
        print(f"Not found: {ckpt_path}")
        return None, None, None, None
    model, vision_encoder, img_proc, tok = build_model(config, projector_type, "cuda", ckpt_path)
    model.eval()
    return model, vision_encoder, img_proc, tok

# 사용: model, ve, img_proc, tok = load_trained_model("E7_v2_perceiver_optimized", "perceiver", CONFIG)